In [1]:
%%html
<b>Press play on the music player(Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [2]:
#@title 1. Install Libraries
!pip install gradio
!python3 -m pip install -U demucs
!pip install yt_dlp
!pip install ffmpeg
!pip install pydub  # Install library pydub
!mkdir youtubeaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 

In [4]:
#@title 2. run app

import os
import gradio as gr


# cvt audio

from pydub import AudioSegment
def convert_audio(url):
  
  # Mendefinisikan path untuk file audio
  input_path = url
  file_name = os.path.basename(input_path)
  filename = os.path.splitext(file_name)[0]
  

  parent_dir = os.path.dirname(url)
  new_path = os.path.relpath(parent_dir, "/content")

 

  output_path = f'{new_path}/{filename}_converted.mp3'

  # Mengkonversi file audio WAV menjadi MP3 menggunakan pydub
  sound = AudioSegment.from_wav(input_path)
  sound.export(output_path, format="mp3")

  # Mengecek apakah file audio MP3 sudah tersimpan
  if os.path.isfile(output_path):
      # return output_path
      return "sukses"
  else:
      return "Konversi gagal"


# Fungsi play Audio
def play_audio(url):

  file_path = url
  file_name = os.path.basename(file_path)
  filename = os.path.splitext(file_name)[0]

  original_path = f"/content/youtubeaudio/{filename}.wav"
  vocal_path = f"/content/separated/htdemucs/{filename}/vocals.wav"
  instrument_path = f"/content/separated/htdemucs/{filename}/no_vocals.wav"

  return url, vocal_path, instrument_path

# Fungsi download audio
from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys

  
def download_audio(title, url):

  ydl_opts = {
    'format': 'bestaudio/best',
    #   'outtmpl': 'output.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    "outtmpl": f'youtubeaudio/{title}',  # this is where you can edit how you'd like the filenames to be formatted
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    # url = "https://www.youtube.com/watch?v=LCcNtQuhUgg" #@param {type:"string"}
    ydl.download([url])
  # return f"/content/youtubeaudio/{title}.wav"
  # return f"/content/youtubeaudio/adudio.wav"
  return "sukses"


# fungsi split audio
def split_audio(url):
  import subprocess

  command = f"demucs --two-stems=vocals {url}"
  result = subprocess.run(command.split(), stdout=subprocess.PIPE)
  print(result.stdout.decode())
  return "sukses"

with gr.Blocks() as demo:
    gr.Markdown("Audio Tools")

    with gr.Tab("Audio Player and Convert"):
        with gr.Row():
          with gr.Column():
            p_input = gr.Textbox(label = "Audio Url")
            p_button = gr.Button("Play")
          with gr.Column():
            p_output =[gr.Audio(label = "Original"), gr.Audio(label = "Vocal"), gr.Audio(label = "Instrumental")]
      

    with gr.Tab("Download Youtube"):
        with gr.Row():
          with gr.Column():
            d_input = [gr.Textbox(label = "title"), gr.Textbox(label = "Youtube Url")]
            d_button = gr.Button("Download")
          with gr.Column():
            d_output =[gr.Textbox(label = "Status")]
      

    with gr.Tab("Split Audio"):
        with gr.Row():
          with gr.Column():
            s_input = gr.Textbox(label = "Audio Url")
            s_button = gr.Button("Split")
          with gr.Column():
            s_output = gr.Textbox(label = "Status")

    with gr.Tab("Convert To MP3"):
        with gr.Row():
          with gr.Column():
            c_input = gr.Textbox(label = "Audio Url")
            c_button = gr.Button("Convert")
          with gr.Column():
            c_output = gr.Textbox(label = "Output")
        
    

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    p_button.click(play_audio, inputs=p_input, outputs=p_output)
    d_button.click(download_audio, inputs=d_input, outputs=d_output)
    s_button.click(split_audio, inputs=s_input, outputs=s_output)
    c_button.click(convert_audio, inputs=c_input, outputs=c_output)

demo.launch()






Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

[![Github Logo](https://i.imgur.com/9I6NRUm.png)](https://github.com/Github_Username)
[firzaelbuho](https://github.com/firzaelbuho)
